In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import extractor
import os
from sklearn import svm

In [2]:
authenticated_users = ['JarupulaBaba','nitheesh','Bharath','Asrith','Pavan']

In [3]:
X_auth=[]

In [4]:
for user in authenticated_users:
    print("extracting features for user "+user)
    if (os.path.exists(user+'.npy')):
        print("present")
        X_auth.append(np.load(user+'.npy'))
    else:
        folder = "data/authenticated/"+user+"/"
        X=extractor.features(len(os.listdir(folder)),folder)
        np.save(user,X)
        X_auth.append(X)

extracting features for user JarupulaBaba
present
extracting features for user nitheesh
present
extracting features for user Bharath
present
extracting features for user Asrith
present
extracting features for user Pavan
present


In [9]:
#for imposter users
if (os.path.exists('imposter.npy')):
    X_imposter=np.load('imposter.npy')
else:
    X_imposter=extractor.features(len(os.listdir("data/imposter/")),"data/imposter/")
    np.save("imposter",X_imposter)
#y_imposter.shape
y_imposter=np.ones(len(X_imposter))

In [10]:
X_auth_final=np.concatenate((X_auth[0],X_auth[1],X_auth[2],X_auth[3],X_auth[4]),axis=0)
y_auth_final=np.zeros(len(X_auth_final))
print(X_auth_final.shape)
print(y_auth_final.shape)

(19139, 63)
(19139,)


In [11]:
#X_train=np.concatenate((X_auth_final[:int(X_auth_final.shape[0]*0.8)],X_imposter[:int(X_imposter.shape[0]*0.8)]),axis=0)
#y_train=np.concatenate((y_auth_final[:int(X_auth_final.shape[0]*0.8)],y_imposter[:int(X_imposter.shape[0]*0.8)]),axis=0)
#X_test=np.concatenate((X_auth_final[int(X_auth_final.shape[0]*0.8):],X_imposter[int(X_imposter.shape[0]*0.8):]),axis=0)
#y_test=np.concatenate((y_auth_final[int(X_auth_final.shape[0]*0.8):],y_imposter[int(y_imposter.shape[0]*0.8):]),axis=0)

In [12]:
X=np.concatenate((X_auth_final,X_imposter),axis=0)
y=np.concatenate((y_auth_final,y_imposter),axis=0)
print(X.shape)
print(y.shape)

(35693, 63)
(35693,)


In [13]:
y[y==1].size

16554

In [14]:
knn = KNeighborsClassifier(n_neighbors=63)
sfs1 = SFS(knn,
           k_features=5,
           forward=True,
           floating=False,
           scoring='accuracy',
           cv=4)

In [ ]:
sfs1 = sfs1.fit(X, y)
X = X[:,sfs1.k_feature_idx_]

In [ ]:
X.shape

In [ ]:
kf = KFold(n_splits=10,shuffle=True)
accuracy_model=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #print(X_train.shape)
    #print(y_train.shape)
    #print(X_test.shape)
    #print(y_test.shape)
    #print(y_test.size)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    accuracy_model.append(accuracy_score(y_test, clf.predict(X_test), normalize=True)*100)
    print(accuracy_score(y_test, clf.predict(X_test), normalize=True)*100)